In [47]:
from label_studio_sdk import Client

# Initialize the client
ls = Client(url='http://10.0.101.10:8080', api_key='9128fab0ec30016774d8ae41fb84f541a7955817')

# Get the project
project = ls.get_project(20)

# Export tasks
tasks = project.export_tasks(export_type='JSON', download_all_tasks=True)

In [22]:
import json
import os

# === CONFIGURACIÓN ===

json_path = "/workspace/RGB_T_SAR/export_v_II/lucas_tir_sam2.json"  # Tu archivo de Label Studio
output_dir = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_lucas_tir/"  # Carpeta donde guardarás los JSON individuales

os.makedirs(output_dir, exist_ok=True)

# === Cargar JSON principal ===
with open(json_path, "r") as f:
    all_tasks = json.load(f)

# === Guardar cada task en un archivo separado ===
for i, task in enumerate(all_tasks):
    output_path = os.path.join(output_dir, f"task_{i+1:04}.json")
    with open(output_path, "w") as out_f:
        json.dump(task, out_f, indent=2)

print(f"✅ Se dividieron {len(all_tasks)} tareas en archivos individuales en:\n{output_dir}")


✅ Se dividieron 20 tareas en archivos individuales en:
/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_lucas_tir/


In [23]:
import json
import os
import numpy as np
from PIL import Image
from label_studio_sdk.converter.brush import decode_rle  # asegúrate de importar esto

# === CONFIGURACIÓN ===

input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_lucas_tir/"  # Carpeta con tus JSON
output_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_to_png_lucas_tir/"
os.makedirs(output_mask_folder, exist_ok=True)

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    task = data[0] if isinstance(data, list) else data
    annotations = task['annotations'][0]['result']

    # Mapas de clases
    label_map = {}
    label_value = 1

    # Detectar altura y anchura a partir de la primera anotación
    first_rle = annotations[0]['value']['rle']
    if isinstance(first_rle, dict):
        size = first_rle['size']
    else:
        size = annotations[0]['original_height'], annotations[0]['original_width']

    height, width = size
    mask = np.zeros((height, width), dtype=np.uint8)

    for ann in annotations:
        if 'value' not in ann:
            print(f"Saltando anotación sin 'value' en {json_file}")
            continue
        if 'brushlabels' not in ann['value']:
            print(f"Saltando anotación sin 'brushlabels' en {json_file}")
            continue

        label = ann['value']['brushlabels'][0]
        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        rle = ann['value']['rle']

        if isinstance(rle, dict):
            decoded_mask = decode_rle(rle['counts'], rle['size'])
        else:
            decoded_mask = decode_rle(rle, (height, width))

        if decoded_mask.size == height * width * 4:
            decoded_mask = decoded_mask.reshape((height, width, 4))
            binary_mask = decoded_mask[:, :, 0]
        elif decoded_mask.size == height * width:
            binary_mask = decoded_mask.reshape((height, width))
        else:
            raise ValueError(f"Shape incompatible: decoded size {decoded_mask.size}")

        mask[binary_mask == 255] = label_map[label]

    # Nombre de salida (basado en el nombre del JSON)
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_mask_folder, f"{base_name}.png")
    Image.fromarray(mask).save(output_path)
    print(f"✅ Máscara guardada: {output_path}")

print("\n✅ Todos los JSON han sido convertidos a PNG.")


RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rl

In [24]:
import json
import os
import numpy as np
from PIL import Image
from label_studio_sdk.converter.brush import decode_rle  # asegúrate de importar esto

# === CONFIGURACIÓN ===

input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_lucas_tir/"  # Carpeta con tus JSON
output_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_to_png_lucas_tir/"
os.makedirs(output_mask_folder, exist_ok=True)

# === Mapa FIJO de clases: nombre (Label Studio) -> ID entero ===
# OJO: los strings deben coincidir EXACTAMENTE con los 'brushlabels' de Label Studio
CLASS_TO_ID = {
    "First-Responder": 1,
    "Civilian": 2,
    "Vegetation": 3,
    "Road": 4,
    "Dirt-Road": 5,
    "Building": 6,
    "Sky": 7,
    "Civilian-Car": 8,
    "Responder-Vehicle": 9,
    "Debris": 10,
    "Command-Post": 11,
}
# 0 queda como fondo/unlabeled

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    task = data[0] if isinstance(data, list) else data
    annotations = task['annotations'][0]['result']

    if len(annotations) == 0:
        print(f"⚠ {json_file} no tiene anotaciones, se salta.")
        continue

    # Detectar altura y anchura a partir de la primera anotación
    first_rle = annotations[0]['value']['rle']
    if isinstance(first_rle, dict):
        size = first_rle['size']  # normalmente [height, width]
        height, width = size[0], size[1]
    else:
        # fallback usando campos originales
        height = annotations[0].get('original_height')
        width = annotations[0].get('original_width')
        if height is None or width is None:
            raise ValueError(f"No se encontraron dimensiones válidas en {json_file}")

    # Inicializar máscara (0 = unlabeled)
    mask = np.zeros((height, width), dtype=np.uint8)

    for ann in annotations:
        if 'value' not in ann:
            print(f"   → Saltando anotación sin 'value' en {json_file}")
            continue
        if 'brushlabels' not in ann['value']:
            print(f"   → Saltando anotación sin 'brushlabels' en {json_file}")
            continue
        if 'rle' not in ann['value']:
            print(f"   → Saltando anotación sin 'rle' en {json_file}")
            continue

        label = ann['value']['brushlabels'][0]

        # Obtener ID de clase a partir del mapa fijo
        class_id = CLASS_TO_ID.get(label, 0)
        if label not in CLASS_TO_ID:
            print(f"⚠ Etiqueta '{label}' no está en CLASS_TO_ID, se marcará como 0 (unlabeled) en {json_file}")

        rle = ann['value']['rle']

        # Decodificar RLE
        if isinstance(rle, dict):
            decoded_mask = decode_rle(rle['counts'], rle['size'])
            h_dec, w_dec = rle['size'][0], rle['size'][1]
        else:
            decoded_mask = decode_rle(rle, (height, width))
            h_dec, w_dec = height, width

        # Manejar posibles formatos: (H, W) o (H, W, 4)
        if decoded_mask.size == h_dec * w_dec * 4:
            decoded_mask = decoded_mask.reshape((h_dec, w_dec, 4))
            binary_mask = decoded_mask[:, :, 0]
        elif decoded_mask.size == h_dec * w_dec:
            binary_mask = decoded_mask.reshape((h_dec, w_dec))
        else:
            raise ValueError(f"Shape incompatible en {json_file}: decoded size {decoded_mask.size}")

        # A veces el brush es 0/1, otras 0/255 → normalizamos
        # Consideramos "pintado" todo pixel > 0
        mask[binary_mask > 0] = class_id

    # Nombre de salida (basado en el nombre del JSON)
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_mask_folder, f"{base_name}.png")
    Image.fromarray(mask).save(output_path)
    print(f"✅ Máscara guardada: {output_path}")

print("\n✅ Todos los JSON han sido convertidos a PNG con IDs de clase fijos.")


RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rl

In [25]:
import json
import os
import numpy as np
from PIL import Image
from label_studio_sdk.converter.brush import decode_rle

# === CONFIGURACIÓN ===
input_json_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_lucas_tir/"
output_color_mask_folder = "/workspace/RGB_T_SAR//proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_to_color_lucas_tir/"
os.makedirs(output_color_mask_folder, exist_ok=True)

# === Diccionario de colores específicos por clase (RGB) ===
class_colors = {
    "First-Responder": (255, 132, 0),
    "Civilian": (224, 67, 216),
    "Vegetation": (65, 117, 5),
    "Road": (155, 155, 155),
    "Dirt-Road": (255, 221, 104),
    "Building": (180, 117, 31),
    "Sky": (22, 152, 207),
    "Civilian-Car": (64, 0, 128),
    "Responder-Vehicle": (192, 64, 0),
    "Debris": (60, 37, 0),
    "Command-Post": (192, 128, 128)
}

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    task = data[0] if isinstance(data, list) else data
    annotations = task['annotations'][0]['result']

    # Mapas de clases
    label_map = {}
    label_value = 1

    # Detectar altura y anchura
    first_rle = annotations[0]['value']['rle']
    if isinstance(first_rle, dict):
        size = first_rle['size']
    else:
        size = annotations[0]['original_height'], annotations[0]['original_width']

    height, width = size
    mask = np.zeros((height, width), dtype=np.uint8)

    for ann in annotations:
        if 'value' not in ann or 'brushlabels' not in ann['value']:
            print(f"Saltando anotación no válida en {json_file}")
            continue

        label = ann['value']['brushlabels'][0]
        if label not in label_map:
            label_map[label] = label_value
            label_value += 1

        rle = ann['value']['rle']
        if isinstance(rle, dict):
            decoded_mask = decode_rle(rle['counts'], rle['size'])
        else:
            decoded_mask = decode_rle(rle, (height, width))

        if decoded_mask.size == height * width * 4:
            decoded_mask = decoded_mask.reshape((height, width, 4))
            binary_mask = decoded_mask[:, :, 0]
        elif decoded_mask.size == height * width:
            binary_mask = decoded_mask.reshape((height, width))
        else:
            raise ValueError(f"Shape incompatible: decoded size {decoded_mask.size}")

        mask[binary_mask == 255] = label_map[label]

    # === Crear y guardar máscara coloreada usando los colores fijos ===
    mask_rgb = np.zeros((height, width, 3), dtype=np.uint8)

    for label_name, label_idx in label_map.items():
        if label_name in class_colors:
            color = np.array(class_colors[label_name])
        else:
            color = np.array([255, 255, 255])  # Color blanco si falta

        mask_rgb[mask == label_idx] = color.astype(np.uint8)

    base_name = os.path.splitext(json_file)[0]
    color_output_path = os.path.join(output_color_mask_folder, f"{base_name}.png")
    Image.fromarray(mask_rgb).save(color_output_path)
    print(f"✅ Máscara coloreada guardada: {color_output_path}")

RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rl

In [5]:
import os
import json

# === CONFIGURACIÓN ===
# Carpeta donde tienes los JSON originales
INPUT_DIR = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_andres_ii_rgb/"

# Carpeta donde quieres guardar los JSON limpios
OUTPUT_DIR = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/estadistica/sam2/json_andres_ii_tir_limpio/"

# Prefijo actual de la ruta de la imagen en "data.image"
# Tal como aparece en tus JSON:
# "data": {
#     "image": "/data/local-files/?d=proyecto_oct_2025_sam2/dataset/dataset_2025/rgb/0001.png"
# }
OLD_IMAGE_PREFIX = "/data/local-files/?d=media/"


# Nuevo prefijo que quieres usar (MODIFICA ESTO A TU GUSTO)
NEW_IMAGE_PREFIX = "/data/local-files/?d=media/Imagenes_tir/"


def clean_task(task: dict) -> dict:
    """Elimina 'id' en el nivel superior y dentro de 'annotations',
    y actualiza la ruta de data.image si corresponde."""
    # Quitar id del nivel superior
    task.pop("id", None)

    # Modificar ruta de la imagen si existe
    try:
        image_path = task["data"]["image"]
        if isinstance(image_path, str) and OLD_IMAGE_PREFIX and image_path.startswith(OLD_IMAGE_PREFIX):
            # Reemplaza solo el prefijo
            task["data"]["image"] = image_path.replace(OLD_IMAGE_PREFIX, NEW_IMAGE_PREFIX, 1)
    except (KeyError, TypeError):
        # Si no hay data o image, lo ignoramos
        pass

    # Limpiar annotations
    annotations = task.get("annotations", [])
    if isinstance(annotations, list):
        for ann in annotations:
            if isinstance(ann, dict):
                ann.pop("id", None)
                ann.pop("unique_id", None)

    return task


def process_file(input_path: str, output_path: str):
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # El export de Label Studio a veces es una lista de tareas, o una sola tarea
    if isinstance(data, list):
        cleaned = [clean_task(task) for task in data if isinstance(task, dict)]
    elif isinstance(data, dict):
        cleaned = clean_task(data)
    else:
        print(f"Formato no esperado en {input_path}, se omite.")
        return

    # Guardar JSON limpio
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(cleaned, f, ensure_ascii=False, indent=2)

    print(f"Procesado: {os.path.basename(input_path)}  ->  {os.path.basename(output_path)}")


def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for fname in os.listdir(INPUT_DIR):
        if not fname.lower().endswith(".json"):
            continue

        in_path = os.path.join(INPUT_DIR, fname)
        out_path = os.path.join(OUTPUT_DIR, fname)

        process_file(in_path, out_path)


if __name__ == "__main__":
    main()




Procesado: task_0001.json  ->  task_0001.json
Procesado: task_0002.json  ->  task_0002.json
Procesado: task_0003.json  ->  task_0003.json
Procesado: task_0004.json  ->  task_0004.json
Procesado: task_0005.json  ->  task_0005.json
Procesado: task_0006.json  ->  task_0006.json
Procesado: task_0007.json  ->  task_0007.json
Procesado: task_0008.json  ->  task_0008.json
Procesado: task_0009.json  ->  task_0009.json
Procesado: task_0010.json  ->  task_0010.json
Procesado: task_0011.json  ->  task_0011.json
Procesado: task_0012.json  ->  task_0012.json
Procesado: task_0013.json  ->  task_0013.json
Procesado: task_0014.json  ->  task_0014.json
Procesado: task_0015.json  ->  task_0015.json
Procesado: task_0016.json  ->  task_0016.json
Procesado: task_0017.json  ->  task_0017.json
Procesado: task_0018.json  ->  task_0018.json
Procesado: task_0019.json  ->  task_0019.json
Procesado: task_0020.json  ->  task_0020.json


In [49]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw
from label_studio_sdk.converter.brush import decode_rle  # <--- IMPORTANTE

# === CONFIGURACIÓN ===

input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/json/"  # Carpeta con tus JSON
output_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/json_to_png/"
os.makedirs(output_mask_folder, exist_ok=True)

# === Mapa de clases FIJO ===
# Asegúrate de que estas cadenas coincidan EXACTAMENTE con los 'polygonlabels' o 'brushlabels' del JSON
CLASS_MAP = {
    "First-Responder": 1,
    "Civilian": 2,
    "Vegetation": 3,
    "Road": 4,
    "Dirt-Road": 5,
    "Building": 6,
    "Sky": 7,
    "Civilian-Car": 8,
    "Responder-Vehicle": 9,
    "Debris": 10,
    "Command-Post": 11,
}
# Cualquier etiqueta no encontrada en CLASS_MAP será "unlabeled" = 0

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith('.json')]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, 'r') as f:
        data = json.load(f)

    # Soporta tanto lista de tareas como un solo dict
    task = data[0] if isinstance(data, list) else data

    # Si no hay anotaciones, omitir este JSON
    if 'annotations' not in task or not task['annotations']:
        print(f"⚠ {json_file}: sin anotaciones, se omite.")
        continue

    # Tomar la primera anotación (puedes adaptarlo si tienes más de una)
    ann0 = task['annotations'][0]

    if 'result' not in ann0 or not ann0['result']:
        print(f"⚠ {json_file}: anotación sin 'result', se omite.")
        continue

    annotations = ann0['result']

    # === Detectar altura y anchura ===
    height = None
    width = None

    # Intentamos encontrar tamaño de imagen examinando las anotaciones
    for ann in annotations:
        # Caso polígonos (Label Studio suele guardar original_height/width en el nivel de ann)
        if height is None and width is None:
            if 'original_height' in ann and 'original_width' in ann:
                height = ann['original_height']
                width = ann['original_width']
                break

    # Si no se encontró, probamos con RLE (brushlabels)
    if height is None or width is None:
        for ann in annotations:
            val = ann.get('value', {})
            if 'rle' in val:
                rle = val['rle']
                if isinstance(rle, dict) and 'size' in rle:
                    # size normalmente [height, width]
                    size = rle['size']
                    height, width = size
                    break
                else:
                    # Si no viene el size en el dict, usar original_height/width
                    if 'original_height' in ann and 'original_width' in ann:
                        height = ann['original_height']
                        width = ann['original_width']
                        break

    if height is None or width is None:
        raise ValueError(f"No se encontraron dimensiones de imagen en {json_file}")

    # Creamos la máscara como imagen PIL para poder mezclar polígonos y brush
    mask_img = Image.new(mode="L", size=(width, height), color=0)
    draw = ImageDraw.Draw(mask_img)

    for ann in annotations:
        if 'value' not in ann:
            print(f"Saltando anotación sin 'value' en {json_file}")
            continue

        val = ann['value']

        # === CASO 1: Polígonos (polygonlabels + points) ===
        if 'polygonlabels' in val and 'points' in val:
            label = val['polygonlabels'][0]
            class_id = CLASS_MAP.get(label, 0)
            if class_id == 0 and label not in CLASS_MAP:
                print(f"⚠️ Etiqueta polígono '{label}' no está en CLASS_MAP; se marcará como 0 (unlabeled) en {json_file}")

            points = val['points']
            # Convertir puntos normalizados (0-100) a píxeles
            poly_pts = [
                (p[0] * width / 100.0, p[1] * height / 100.0)
                for p in points
            ]

            # Dibujar el polígono lleno con el valor de la clase
            draw.polygon(poly_pts, outline=class_id, fill=class_id)

        # === CASO 2: Brush (brushlabels + rle) ===
        elif 'brushlabels' in val and 'rle' in val:
            label = val['brushlabels'][0]
            class_id = CLASS_MAP.get(label, 0)
            if class_id == 0 and label not in CLASS_MAP:
                print(f"⚠️ Etiqueta brush '{label}' no está en CLASS_MAP; se marcará como 0 (unlabeled) en {json_file}")

            rle = val['rle']

            # Decodificar RLE con decode_rle
            if isinstance(rle, dict):
                decoded_mask = decode_rle(rle['counts'], rle['size'])
                h_rle, w_rle = rle['size']
            else:
                decoded_mask = decode_rle(rle, (height, width))
                h_rle, w_rle = height, width

            # Ajustar forma
            if decoded_mask.size == h_rle * w_rle * 4:
                decoded_mask = decoded_mask.reshape((h_rle, w_rle, 4))
                binary_mask = decoded_mask[:, :, 0]
            elif decoded_mask.size == h_rle * w_rle:
                binary_mask = decoded_mask.reshape((h_rle, w_rle))
            else:
                raise ValueError(f"Shape incompatible en {json_file}: decoded size {decoded_mask.size}")

            # Normalizar a 0/255 (puede venir 0/1, 0/255, etc.)
            binary_mask = (binary_mask > 0).astype(np.uint8) * 255

            # Creamos máscara PIL de la brocha
            brush_mask_img = Image.fromarray(binary_mask, mode="L")

            # Pegamos el valor de clase donde la máscara sea no cero
            mask_img.paste(class_id, mask=brush_mask_img)

        else:
            # Si no es ni polígono ni brush, lo ignoramos
            print(f"ℹ️ Anotación en {json_file} no es ni polygonlabels ni brushlabels, se ignora.")

    # Convertir la máscara PIL a numpy (por si luego quieres procesarla)
    mask = np.array(mask_img, dtype=np.uint8)

    # Nombre de salida (basado en el nombre del JSON)
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_mask_folder, f"{base_name}.png")
    Image.fromarray(mask).save(output_path)
    print(f"✅ Máscara (polígonos + brush) guardada: {output_path}")

print("\n✅ Todos los JSON han sido convertidos a PNG con soporte para polygonlabels y brushlabels.")


RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rl

In [50]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw
from label_studio_sdk.converter.brush import decode_rle  # IMPORTANTE

# === CONFIGURACIÓN ===
input_json_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/json/"
output_color_mask_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/json_to_color_png/"
os.makedirs(output_color_mask_folder, exist_ok=True)

# === Paleta de colores por clase (RGB) ===
class_colors = {
    "First-Responder": (255, 132, 0),
    "Civilian": (224, 67, 216),
    "Vegetation": (65, 117, 5),
    "Road": (155, 155, 155),
    "Dirt-Road": (255, 221, 104),
    "Building": (180, 117, 31),
    "Sky": (22, 152, 207),
    "Civilian-Car": (64, 0, 128),
    "Responder-Vehicle": (192, 64, 0),
    "Debris": (60, 37, 0),
    "Command-Post": (192, 128, 128),
}

# Color para unlabeled (etiquetas no encontradas)
UNLABELED_COLOR = (0, 0, 0)

# === Procesar todos los archivos JSON ===
json_files = [f for f in os.listdir(input_json_folder) if f.endswith(".json")]

for json_file in json_files:
    json_path = os.path.join(input_json_folder, json_file)

    with open(json_path, "r") as f:
        data = json.load(f)

    # Soporta tanto lista de tareas como un solo dict
    task = data[0] if isinstance(data, list) else data

    # Si no hay anotaciones, omitir
    if "annotations" not in task or not task["annotations"]:
        print(f"⚠ {json_file}: sin anotaciones, se omite.")
        continue

    ann0 = task["annotations"][0]

    if "result" not in ann0 or not ann0["result"]:
        print(f"⚠ {json_file}: anotación sin 'result', se omite.")
        continue

    annotations = ann0["result"]

    # === Detectar altura y anchura ===
    height = None
    width = None

    # 1) Buscar original_height/width
    for ann in annotations:
        if height is None and width is None:
            if "original_height" in ann and "original_width" in ann:
                height = ann["original_height"]
                width = ann["original_width"]
                break

    # 2) Si no se encontró, intentar con RLE size
    if height is None or width is None:
        for ann in annotations:
            val = ann.get("value", {})
            if "rle" in val:
                rle = val["rle"]
                if isinstance(rle, dict) and "size" in rle:
                    height, width = rle["size"]  # [h, w]
                    break
                elif "original_height" in ann and "original_width" in ann:
                    height = ann["original_height"]
                    width = ann["original_width"]
                    break

    if height is None or width is None:
        raise ValueError(f"No se encontraron dimensiones de imagen en {json_file}")

    # === Crear máscara a color ===
    color_mask_img = Image.new(mode="RGB", size=(width, height), color=UNLABELED_COLOR)
    draw = ImageDraw.Draw(color_mask_img)

    for ann in annotations:
        if "value" not in ann:
            print(f"Saltando anotación sin 'value' en {json_file}")
            continue

        val = ann["value"]

        # === CASO 1: Polígonos ===
        if "polygonlabels" in val and "points" in val:
            label = val["polygonlabels"][0]
            color = class_colors.get(label, UNLABELED_COLOR)

            if label not in class_colors:
                print(f"⚠️ Etiqueta polígono '{label}' no está en class_colors; se pintará UNLABELED en {json_file}")

            points = val["points"]
            poly_pts = [(p[0] * width / 100.0, p[1] * height / 100.0) for p in points]

            draw.polygon(poly_pts, outline=color, fill=color)

        # === CASO 2: Brush (RLE) ===
        elif "brushlabels" in val and "rle" in val:
            label = val["brushlabels"][0]
            color = class_colors.get(label, UNLABELED_COLOR)

            if label not in class_colors:
                print(f"⚠️ Etiqueta brush '{label}' no está en class_colors; se pintará UNLABELED en {json_file}")

            rle = val["rle"]

            # Decodificar RLE
            if isinstance(rle, dict):
                decoded_mask = decode_rle(rle["counts"], rle["size"])
                h_rle, w_rle = rle["size"]
            else:
                decoded_mask = decode_rle(rle, (height, width))
                h_rle, w_rle = height, width

            # Ajustar forma
            if decoded_mask.size == h_rle * w_rle * 4:
                decoded_mask = decoded_mask.reshape((h_rle, w_rle, 4))
                binary_mask = decoded_mask[:, :, 0]
            elif decoded_mask.size == h_rle * w_rle:
                binary_mask = decoded_mask.reshape((h_rle, w_rle))
            else:
                raise ValueError(f"Shape incompatible en {json_file}: decoded size {decoded_mask.size}")

            # Normalizar a máscara binaria 0/255
            binary_mask = (binary_mask > 0).astype(np.uint8) * 255
            brush_mask_img = Image.fromarray(binary_mask, mode="L")

            # Pintar con color donde la máscara sea 1
            color_mask_img.paste(color, mask=brush_mask_img)

        else:
            print(f"ℹ️ Anotación en {json_file} no es ni polygonlabels ni brushlabels, se ignora.")

    # === Guardar salida ===
    base_name = os.path.splitext(json_file)[0]
    output_path = os.path.join(output_color_mask_folder, f"{base_name}.png")
    color_mask_img.save(output_path)
    print(f"✅ Máscara a COLOR guardada: {output_path}")

print("\n✅ Todos los JSON han sido convertidos a PNG a color (RGB) con soporte para polygonlabels y brushlabels.")


RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rle_sizes
RLE params: 1622016 values 8 word_size [3, 4, 8, 16] rl

In [55]:
import os
from PIL import Image

# =========================
# CONFIGURACIÓN
# =========================
folder_1 = "/workspace/RGB_T_SAR/attempt_forth/dataset/merged//"          # ~135 imágenes (jpg/png/jpeg)
folder_2 = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/superimpose/"          # >=171 imágenes (png; puede tener más)
output_folder = "/workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/articulo_mdpi/merged/" # 0001.png ... 0306.png

N1 = 135   # cantidad que quieres tomar de carpeta 1
N2 = 171   # cantidad que quieres tomar de carpeta 2

ALLOWED_1 = {".jpg", ".jpeg", ".png"}   # extensiones válidas para carpeta 1
ALLOWED_2 = {".png"}                    # extensiones válidas para carpeta 2

os.makedirs(output_folder, exist_ok=True)

# =========================
# UTILIDADES
# =========================
def list_files(folder, allowed_exts):
    files = []
    for f in os.listdir(folder):
        ext = os.path.splitext(f)[1].lower()
        if ext in allowed_exts:
            files.append(f)
    return sorted(files)

def save_as_png(img_path, out_path):
    # Asegura PNG consistente (RGB). Si necesitas preservar alpha, dime y lo ajusto.
    img = Image.open(img_path).convert("RGB")
    img.save(out_path)

# =========================
# 1) Seleccionar archivos
# =========================
files_1 = list_files(folder_1, ALLOWED_1)
files_2 = list_files(folder_2, ALLOWED_2)

if len(files_1) < N1:
    raise ValueError(f"Carpeta 1 tiene {len(files_1)} archivos válidos, pero se requieren {N1}.")

if len(files_2) < N2:
    raise ValueError(f"Carpeta 2 tiene {len(files_2)} PNG, pero se requieren {N2}.")

files_1 = files_1[:N1]   # toma exactamente 135
files_2 = files_2[:N2]   # toma exactamente 171 (aunque haya más)

# =========================
# 2) Fusionar y renombrar
# =========================
counter = 1

# primero carpeta 1
for fname in files_1:
    in_path = os.path.join(folder_1, fname)
    out_name = f"{counter:04d}.png"
    out_path = os.path.join(output_folder, out_name)
    save_as_png(in_path, out_path)
    counter += 1

# luego carpeta 2
for fname in files_2:
    in_path = os.path.join(folder_2, fname)
    out_name = f"{counter:04d}.png"
    out_path = os.path.join(output_folder, out_name)
    save_as_png(in_path, out_path)
    counter += 1

print(f"✅ Listo. Total guardadas: {counter-1}")
print(f"📁 Salida: {output_folder}")
print(f"🔢 Rango: 0001.png ... {counter-1:04d}.png")


✅ Listo. Total guardadas: 306
📁 Salida: /workspace/RGB_T_SAR/proyecto_oct_2025_sam2/dataset/dataset_2025/articulo_mdpi/merged/
🔢 Rango: 0001.png ... 0306.png
